In [1]:
import requests 
from tqdm.auto import tqdm
from fastembed import TextEmbedding
import numpy as np
from qdrant_client import QdrantClient, models

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
EMBEDDING_DIMENSIONALITY = 512
model_handle = "jinaai/jina-embeddings-v2-small-en"



In [3]:
query='I just discovered the course. Can I join now?'

embedding_model = TextEmbedding(model_name=model_handle)


embeddings_generator = embedding_model.embed(query)  # reminder this is a generator
embeddings_query = list(embedding_model.embed(query))
  # you can also convert the generator to a list, and that to a numpy array
len(embeddings_query[0]) # Vector of 384 dimensions

Fetching 5 files: 100%|██████████| 5/5 [00:12<00:00,  2.56s/it]


512

In [4]:
print(min(embeddings_query[0]))



-0.11726373885183883


In [5]:
doc = 'Can I still join the course after the start date?'

In [6]:
embeded_doc = list(embedding_model.embed(doc))

In [7]:
embeddings_query[0].dot(embeded_doc[0])

np.float64(0.9008528895674548)

In [8]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [9]:


embedded_doc = [list(embedding_model.embed(doc['text']))[0] for doc in documents]
embedded_doc_np = np.array(embedded_doc)


In [10]:
embedded_doc_np.dot(embeddings_query[0])  # dot product with the query embedding

array([0.76296845, 0.81823782, 0.80853974, 0.71330788, 0.73044992])

In [11]:
for doc in documents:
    doc['full_text'] = doc['question'] + ' ' + doc['text']

In [12]:
embedded_doc = [list(embedding_model.embed(doc['full_text']))[0] for doc in documents]
embedded_doc_np = np.array(embedded_doc)

In [13]:
embedded_doc_np.dot(embeddings_query[0])

array([0.85145432, 0.8436594 , 0.84082872, 0.77551577, 0.80860079])

In [14]:
min_dim = [model["dim"] for model in TextEmbedding.list_supported_models()]

In [15]:
min(min_dim)

384

In [16]:

for model in TextEmbedding.list_supported_models():
    if model['model']=='BAAI/bge-small-en':
        print(model['dim'])

384


In [17]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        doc['full_text'] = doc['question'] + ' ' + doc['text']
        documents.append(doc)

In [18]:

collection_name = "zoomcamp-homework"
qd_client = QdrantClient("http://localhost:6333")


In [19]:
qd_client.delete_collection(collection_name)

True

In [20]:
qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,  
        distance=models.Distance.COSINE  
    )
)

True

In [21]:
points = []
id = 0

for doc in documents:

    point = models.PointStruct(
        id=id,
        vector=models.Document(text=doc['full_text'], model=model_handle), #embed text locally with "jinaai/jina-embeddings-v2-small-en" from FastEmbed
        payload={
            "full_text": doc['full_text'],
            "text": doc['text'],
            "section": doc['section'],
            "course": doc['course']
        } #save all needed metadata fields
    )
    points.append(point)

    id += 1

In [22]:

qd_client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [23]:

def search(query, limit=1):

    results = qd_client.query_points(
        collection_name=collection_name,
        query=models.Document( #embed the query text locally with "jinaai/jina-embeddings-v2-small-en"
            text=query,
            model=model_handle 
        ),
        limit=limit, # top closest matches
        with_payload=True #to get metadata in the results
    )

    return results

In [24]:
result = search(query)

In [25]:
result.points[0].score

0.8620738